In [ ]:
# %%
# -*- coding: utf-8 -*-
"""
Main code for runnning all functions and making output files to be used for analysis. 
It also calculates the emissions and investment for electrification.
"""

import pandas as pd
import numpy as np
import os
import xarray as xr 
from scipy import interpolate
import matplotlib.pyplot as plt


os.getcwd()
DIR = "X:\\user\\zapatacasv\\sdg7\\scripts"   # Set the running directory here
os.chdir(DIR)
import data_import as dat #import own codes after defining home dir
import elec_systems as es
ec = es.ec
gph = es.gph
en = es.en
er = es.er


In [ ]:
# ##WHEN RUNNING IN PARTS

# import pandas as pd
# import numpy as np
# import os
# import xarray as xr 
# from scipy import interpolate
# import matplotlib.pyplot as plt


# os.getcwd()
# DIR = "X:\\user\\zapatacasv\\sdg7\\scripts"   # Set the running directory here
# os.chdir(DIR)
# import data_import as dat #import own codes after defining home dir
# import test as es
# import test as gph
# import test as er
# import test as en
# es.s = xr.open_dataset('..\\output\\es_s_world.nc').sel(year = slice(dat.t0, dat.tf))
# en.capcost= xr.open_dataset('..\\output\\SSP2\\en_capcost_world.nc').sel(year = slice(dat.t0, dat.tf))
# gph.hhnumberg = xr.open_dataset('..\\output\\gph_SSP2\\gph_hhnumberg_world_SSP2.nc').sel(year = slice(dat.t0, dat.tf))
# gph.Elecuseg = xr.open_dataarray('..\\output\\gph_SSP2\\gph_Elecuseg_world_SSP2.nc').sel(year = slice(dat.t0, dat.tf))
# gph.popg = xr.open_dataset('..\\output\\gph_SSP2\\gph_popg_world_SSP2.nc')['tpop'].sel(year = slice(dat.t0, dat.tf))
# er.elec_rate = xr.open_dataset('..\\output\\er_SSP2_world_annual.nc').sel(year = slice(dat.t0, dat.tf))





In [ ]:
#Cleaning data from countries that achieved UA when running for world
#countries with UA by 2020
path = "..\\data\\UA_countries_2020.csv"
df2 =  pd.read_csv(path,  delimiter = ',')
#loading country  mask and removing countries that achieved UA
df = xr.open_dataset('..\\data\\country_5min.nc')
df = df.assign_coords(x = ("x", gph.popg.x.values))
df = df.assign_coords(y = ("y", gph.popg.y.values)) 
gph.popg.coords['country'] = xr.where(df.country.isin(df2.id), np.nan, df.country)
gph.popg['country'] = gph.popg.country.where((gph.popg.country != 255) & (gph.popg.country != 8) & (gph.popg.country != 88) & (gph.popg.country != 214)) #remove oceans and poles for country code

gph.popg = gph.popg.where(gph.popg['country']>=0)
es.s = es.s.where(gph.popg['country']>=0)
en.capcost = en.capcost.where(gph.popg['country']>=0)
gph.Elecuseg = gph.Elecuseg.where(gph.popg['country']>=0)
gph.hhnumberg = gph.hhnumberg.where(gph.popg['country']>=0)
gph.hhnumberg = gph.hhnumberg.hhnumberg


In [ ]:
################Investment baseline scenario
nrc =  dat.nrc #[8,9,10,26]#
#Annualized total cost for electricity access under baseline
ElecConsum_PerHH = xr.full_like(gph.Elecuseg, np.nan) 
for r in nrc: 
    ElecConsum_PerHH = xr.where(gph.Elecuseg.imager == r, dat.elec_use_perhh.sel(year = gph.hhnumberg.year.data, region = r,  turq=1), ElecConsum_PerHH) #[kWh]

year = es.s.LeastCostElecCOST.year.data
leastcost_LCOE_enteryear = xr.full_like(es.s.LeastCostElecCOST.imager, np.nan)    #For selecting the LCOE per grid cell depending on the year the cell gets access
techid_bl = xr.full_like(es.s.LeastCostElecCOST.imager, np.nan)  #For defining the technology per grid cell depending on the year the cell gets access
hh_increase = xr.full_like(gph.Elecuseg, 0)
marg_elecuseg = xr.full_like(gph.Elecuseg, 0) 
increase_elecuse_perhh = xr.full_like(gph.Elecuseg, np.nan)
for y in range(len(year)): 
    leastcost_LCOE_enteryear = xr.where(es.s.new_access_BL.sel(year = year[y]) ==1, es.s.LeastCostElecCOST.sel(year = year[y]), leastcost_LCOE_enteryear)
    techid_bl = xr.where(es.s.new_access_BL.sel(year = year[y]) ==1, es.s.LeastCostTechID.sel(year = year[y]), techid_bl) 

for y in range(len(year)):
    if y !=0:
        marg_elecuseg[dict(year= y)] = np.maximum(0, gph.Elecuseg.sel(year = year[y]) - gph.Elecuseg.sel(year = year[y-1]))
        hh_increase[dict(year= y)] = np.maximum(0, gph.hhnumberg.sel(year = year[y],  turq=1) - gph.hhnumberg.sel(year = year[y-1],  turq=1)) 
        increase_elecuse_perhh[dict(year= y)] = np.maximum(0, ElecConsum_PerHH.sel(year = year[y]) - ElecConsum_PerHH.sel(year = year[y-1])) 
        
AnnGridInv_BL =  xr.full_like(gph.Elecuseg, np.nan) 
AnnGridInv_BL = xr.where(techid_bl>1, leastcost_LCOE_enteryear*gph.Elecuseg, AnnGridInv_BL)  
AnnGridInv_BL = xr.where(techid_bl==1, (leastcost_LCOE_enteryear*increase_elecuse_perhh*gph.hhnumberg.sel( year = dat.t0,  turq=1)) + (hh_increase*en.capcost.TDCapCost_perHHg), AnnGridInv_BL)  #! For old connection but new demand + marginal investment for extra production + For new connections on existing power lines


for r in nrc:
    AnnGridIAnnGridInv_BLv_univ = xr.where((AnnGridInv_BL.imager == r)&(er.elec_rate.rural.sel(region = r, year = dat.cal_year) > 0.99), np.nan, AnnGridInv_BL)   #for removing areas where there was already access by 2020




cum_sum_investment_BL = AnnGridInv_BL.cumsum(dim = 'year', skipna=True)
investment_region_BL = cum_sum_investment_BL.sel(year = dat.tf).groupby("imager").sum()  
AnnGridInv_region_BL = AnnGridInv_BL.groupby("imager").sum()

print("investment_region_BL [billion$] is:", investment_region_BL.sum().values/1e9)
print('average price electricity BL2 [$/MWh] is:', leastcost_LCOE_enteryear.weighted(gph.Elecuseg.sel(year = dat.tf).fillna(0)).mean().values*1e3 )


In [ ]:
#Calculating baseline investment for SSA 
print("investment_SSA_BL [billion$] is:", investment_region_BL.where((investment_region_BL.imager == 8) | (investment_region_BL.imager == 9) | (investment_region_BL.imager == 10) | (investment_region_BL.imager == 26), drop = True).sum().values/1e9)
print('average price electricity BL SSA [$/MWh] is:', leastcost_LCOE_enteryear.where((leastcost_LCOE_enteryear.imager == 8) | (leastcost_LCOE_enteryear.imager == 9) | (leastcost_LCOE_enteryear.imager == 10) | (leastcost_LCOE_enteryear.imager == 26), drop = True).weighted(gph.Elecuseg.sel(year = dat.tf).fillna(0)).mean().values*1e3 )


In [ ]:
#########Investment under universal access
#Annualized total cost for electricity access under universal access

leastcost_LCOE_enteryear_univ = xr.full_like(es.s.LeastCostElecCOST.imager, np.nan)  
techid_univ = xr.full_like(es.s.LeastCostElecCOST.imager, np.nan) 
for y in range(len(year)):  #between 2010 to dat.tf
    leastcost_LCOE_enteryear_univ = xr.where(es.s.new_access_univ.sel(year = year[y]) ==1, es.s.LeastCostElecCOST.sel(year = year[y]), leastcost_LCOE_enteryear_univ)
    techid_univ = xr.where(es.s.new_access_univ.sel(year = year[y]) ==1, es.s.LeastCostTechID.sel(year = year[y]), techid_univ) 

AnnGridInv_univ =  xr.full_like(gph.Elecuseg, np.nan)  

AnnGridInv_univ = xr.where((techid_univ>1), leastcost_LCOE_enteryear_univ*gph.Elecuseg, AnnGridInv_univ) #
AnnGridInv_univ = xr.where(techid_univ==1,  (leastcost_LCOE_enteryear*increase_elecuse_perhh*gph.hhnumberg.sel( year = dat.t0,  turq=1)) + (hh_increase*en.capcost.TDCapCost_perHHg), AnnGridInv_univ)  #  For old connection but new demand, mariginal investment for extra production #Note that the marginal generation cost is assumed to be equal the system_total_cost_perkwh (from TIMER)

for r in nrc:
    AnnGridInv_univ = xr.where((AnnGridInv_univ.imager == r)&(er.elec_rate.rural.sel(region = r, year = dat.cal_year) > 0.99), np.nan, AnnGridInv_univ)   #for removing areas where there was already access

cum_sum_investment_univ = AnnGridInv_univ.cumsum(dim = 'year')
investment_region_univ = cum_sum_investment_univ.sel(year = dat.tf).groupby("imager").sum()
AnnGridInv_region_univ = AnnGridInv_univ.groupby("imager").sum()

print("investment_region_UA [billion$] is:", investment_region_univ.sum().values/1e9)
print('average price electricity UNIV [$/MWh] is:', leastcost_LCOE_enteryear_univ.weighted(gph.Elecuseg.sel(year = dat.tf).fillna(0)).mean().values*1e3 )

In [ ]:
#Saving files of technologies and least LCOE obtained depending on the year each cell gets access
techid_bl.to_netcdf('..\\output\\techid_bl.nc')
techid_univ.to_netcdf('..\\output\\techid_univ.nc')
leastcost_LCOE_enteryear_univ.to_netcdf('..\\output\\leastcost_LCOE_enteryear_univ.nc')
leastcost_LCOE_enteryear.to_netcdf('..\\output\\leastcost_LCOE_enteryear_bl.nc')
AnnGridInv_region_univ.to_netcdf('..\\output\\AnnGridInv_region_univ.nc')
AnnGridInv_region_BL.to_netcdf('..\\output\\AnnGridInv_region_BL.nc')


In [ ]:
#UA investment for SSA
print("investment_SSA_UA [billion$] is:", investment_region_univ.where((investment_region_univ.imager == 8) | (investment_region_univ.imager == 9) | (investment_region_univ.imager == 10) | (investment_region_univ.imager == 26), drop = True).sum().values/1e9)
print('average price electricity UNIV SSA [$/MWh] is:', leastcost_LCOE_enteryear_univ.where((leastcost_LCOE_enteryear_univ.imager == 8) | (leastcost_LCOE_enteryear_univ.imager == 9) | (leastcost_LCOE_enteryear_univ.imager == 10) | (leastcost_LCOE_enteryear_univ.imager == 26), drop = True).weighted(gph.Elecuseg.sel(year = dat.tf).fillna(0)).mean().values*1e3)


In [ ]:
#Average LCOE per technology
leastcost_LCOE_enteryear_univ.coords['techid_univ'] = techid_univ
def fun(x): 
    return (x/x.sum())
weights = gph.Elecuseg.sel(year = dat.tf).fillna(0)
weights['techid_univ'] = techid_univ
weights = weights.groupby('techid_univ').map(fun)
a = leastcost_LCOE_enteryear_univ*weights
a = a.groupby('techid_univ').sum().to_dataframe(name='lcoe')  #weighted mean on consumption........
a.lcoe.to_csv('..\\output\\weighted_LCOE_univ_pertech.csv', index=True)
a.lcoe


In [ ]:
#Emissions baseline  scenario 
e = gph.Elecuseg.to_dataset(name = 'elec_use')
e['elec_use'] = e['elec_use'].where(es.s.new_access_BL >0)  #where there is access for each year
e.coords['id_bl'] = (('y', 'x'), techid_bl.values)
e = e.sel(year = [dat.target_year, dat.cal_year])
e['elec_use'] = e.elec_use #- e.elec_use.sel(year = dat.cal_year)
e['elec_use'] = e['elec_use'].sel(year = dat.target_year)

e['elec_use_r'] =  xr.full_like(er.elec_rate.total.sel(region = nrc,year = dat.target_year), np.nan).expand_dims({'techid_bl': e.id_bl.groupby('id_bl').sum().id_bl.values}).copy()
for r in range(len(nrc)):
    e['elec_use_r'][dict(region= r)]  = e.elec_use.where(e.imager  == nrc[r]).groupby('id_bl').sum().transpose()  #aggregating electricity use per region and grouped by technology


e['elec_use_r'] = e['elec_use_r'].where(e['elec_use_r'] > 0)
e['co2e_bl'] = xr.full_like(e['elec_use_r'], np.nan) 
e['co2e_bl'] = xr.where((e.techid_bl ==1 )| (e.techid_bl ==2) | (e.techid_bl ==3), e['elec_use_r']*dat.CCelec.sel(year = dat.target_year)/dat.GJ2KWh , e['co2e_bl']) #Calculating emissions for new consumption from the grid
e['co2e_bl'] = xr.where((e.techid_bl ==11) | (e.techid_bl ==5), e['elec_use_r']*dat.DG_FuelConsumpperkwh[0]*dat.co2_ldiesel, e['co2e_bl']) #Calculating emissions for new consumption on SA and MG diesel  
e['co2e_bl'] = xr.where((e.techid_bl ==8), e['elec_use_r']*dat.DG_FuelConsumpperkwh[0]*dat.co2_ldiesel*0.3, e['co2e_bl']) #Calculating emissions for new consumption on MG PV- diesel  
e['co2e_bl'] = xr.where((e.techid_bl ==9), e['elec_use_r']*dat.DG_FuelConsumpperkwh[0]*dat.co2_ldiesel*0.5, e['co2e_bl']) #Calculating emissions for new consumption on MG wind- diesel 

e['co2e_bl2'] = e['co2e_bl'].sum(dim = 'techid_bl')

print("co2e_bl in dat.tf in Mton are:", e['co2e_bl2'].sum().values/1e9) #[Mton]
print("share of CO2 emissions by electrification world is : ", e['co2e_bl'].sel(techid_bl =slice(2,11)).sum().values/e['co2e_bl2'].sum().values)
#Electricity use in dat.tf
print("Electricity use baseline in dat.tf in TWh: ", e['elec_use'].sum().values/1e9)

# share of electricity use from renewables in dat.tf
e['renshare']  =  xr.full_like(e['co2e_bl'], np.nan).expand_dims({'dos': np.arange(1,3)}).copy() #1 for CG renewable share and 2 for offgrid ren share
e['renshare'][dict(dos= 0)] =e['elec_use_r'].where((e.techid_bl==1) |(e.techid_bl==2)|(e.techid_bl==3)).sum(dim = 'techid_bl')*dat.ElecProdTotShare.sel(esource = [1,2,3,4,13,14], year = dat.target_year, region = e.region).sum(dim = 'esource') #Calculating power mix from renawables sources for CG
e['renshare'][dict(dos= 1)] = e['elec_use_r'].where((e.techid_bl ==4) | (e.techid_bl ==6) | (e.techid_bl ==7) | (e.techid_bl ==10)).sum(dim = 'techid_bl')

e['renshare_bl']  = e['renshare'].sum(dim = 'dos')/e['elec_use_r'].sum(dim = 'techid_bl')

print('the share of renewables for dat.tf BL is: ' , e['renshare_bl'].mean().values*100)

#Pop
CG = gph.popg.tpop.sel(year = dat.target_year).where( (techid_bl==2)| (techid_bl==3)).sum().values/1e6  #new connections 
SAMG = gph.popg.tpop.sel(year = dat.target_year).where(techid_bl>3).sum().values/1e6
print('million people connected to CG:' , CG)
print('million people connected to offgrid:' , SAMG)




In [ ]:
print("Africa co2e_bl in dat.tf in Mton are:", e['co2e_bl2'].sel(region = [8,9,10,26]).sum().values/1e9) #[Mton]
#Electricity use in dat.tf
print("Africa Electricity use baseline in dat.tf in TWh: ", e['elec_use_r'].sel(region = [8,9,10,26]).sum().values/1e9)
print("share of CO2 emissions by electrification bl SSA is : ", e['co2e_bl'].sel(techid_bl =slice(2,11), region = [8,9,10,26]).sum().values/e['co2e_bl2'].sel(region = [8,9,10,26]).sum().values)
print('the share of renewables for dat.tf BL Africa is: ' , e['renshare_bl'].sel(region = [8,9,10,26]).mean().values*100)

afr = gph.popg.tpop.where((gph.popg.imager == 8) | (gph.popg.imager == 9) | (gph.popg.imager== 10) | (gph.popg.imager == 26), drop = True)

CG = afr.sel(year = dat.target_year).where((techid_bl==1)| (techid_bl==2)| (techid_bl==3)).sum().values/1e6
SAMG = afr.sel(year = dat.target_year).where(techid_bl>3).sum().values/1e6
print('Africa million people connected to CG:' , CG)
print('Africa million people connected to offgrid:' , SAMG)
#saving emissions 
e = e[['co2e_bl2', 'renshare_bl']]
e.to_netcdf('..\\output\\co2emissions_bl_kg.nc')

In [ ]:
#Emissions UA  scenario
#Calculating electricity use where there is access for each year
e = gph.Elecuseg.to_dataset(name = 'elec_use')
e['elec_use'] = e['elec_use'].where(es.s.new_access_univ >0)  #where there is access for each year
e.coords['id_univ'] = (('y', 'x'), techid_univ.values)
e = e.sel(year = [dat.target_year, dat.cal_year])
e['elec_use'] = e['elec_use'].sel(year = dat.target_year)


e['elec_use_r'] =  xr.full_like(er.elec_rate.total.sel(region = nrc,year = dat.target_year), np.nan).expand_dims({'techid_univ': e.id_univ.groupby('id_univ').sum().id_univ.values}).copy()
for r in range(len(nrc)):
    e['elec_use_r'][dict(region= r)]  = e.elec_use.where(e.imager  == nrc[r]).groupby('id_univ').sum().transpose()  #aggregating to regional delta elec_use

e['elec_use_r'] = e['elec_use_r'].where(e['elec_use_r'] > 0)
e['co2e_univ'] = xr.full_like(e['elec_use_r'], np.nan) 
e['co2e_univ'] = xr.where((e.techid_univ ==1 )| (e.techid_univ ==2) | (e.techid_univ ==3), e['elec_use_r']*dat.CCelec.sel(year = dat.target_year)/dat.GJ2KWh , e['co2e_univ']) #Claculating emissions for new consumption from the grid

e['co2e_univ'] = xr.where((e.techid_univ ==11) | (e.techid_univ ==5), e['elec_use_r']*dat.DG_FuelConsumpperkwh[0]*dat.co2_ldiesel, e['co2e_univ']) #Calculating emissions for new consumption on SA and MG diesel  
e['co2e_univ'] = xr.where((e.techid_univ ==8), e['elec_use_r']*dat.DG_FuelConsumpperkwh[0]*dat.co2_ldiesel*0.3, e['co2e_univ']) #Calculating emissions for new consumption on MG PV- diesel  
e['co2e_univ'] = xr.where((e.techid_univ ==9), e['elec_use_r']*dat.DG_FuelConsumpperkwh[0]*dat.co2_ldiesel*0.5, e['co2e_univ']) #Calculating emissions for new consumption on MG wind- diesel 
e['co2e_univ2'] = e['co2e_univ'].sum(dim = 'techid_univ')

#Electricity use and emissions in dat.tf
print("co2_univ in dat.tf in Mton are:", e['co2e_univ2'].sum().values/1e9)  #[Mton]
print("Electricity use universal in dat.tf in TWh : ", e['elec_use'].sum().values/1e9)
print("share of CO2 emissions by electrification is : ", e['co2e_univ'].sel(techid_univ =slice(2,11)).sum().values/e['co2e_univ2'].sum().values)

# share of electricity use from renewables in dat.tf
e['renshare']  =  xr.full_like(e['co2e_univ'], np.nan).expand_dims({'dos': np.arange(1,3)}).copy() #1 for CG renewable share and 2 for offgrid ren share
e['renshare'][dict(dos= 0)] =e['elec_use_r'].where((e.techid_univ==1) |(e.techid_univ==2)|(e.techid_univ==3)).sum(dim = 'techid_univ')*dat.ElecProdTotShare.sel(esource = [1,2,3,4,13,14], year = dat.target_year, region = e.region).sum(dim = 'esource') #Calculating power mix from renawables sources for CG
e['renshare'][dict(dos= 1)] = e['elec_use_r'].where((e.techid_univ ==4) | (e.techid_univ ==6) | (e.techid_univ ==7) | (e.techid_univ ==10)).sum(dim = 'techid_univ')
e['renshare_univ']  = e['renshare'].sum(dim = 'dos')/e['elec_use_r'].sum(dim = 'techid_univ')
print('the share of renewables for dat.tf univ is: ' , e['renshare_univ'].mean().values*100)

#Pop
CG = gph.popg.tpop.sel(year = dat.target_year).where( (techid_univ==2)| (techid_univ==3)).sum().values/1e6
SAMG = gph.popg.tpop.sel(year = dat.target_year).where(techid_univ>3).sum().values/1e6

print('million people connected to CG:' , CG)
print('million people connected to offgrid:' , SAMG)




In [ ]:
#For SSA
print("Africa co2e_univ in dat.tf in Mton are:", e['co2e_univ2'].sel(region = [8,9,10,26]).sum().values/1e9) #[Mton]
print("share of CO2 emissions by electrification bl SSA is : ", e['co2e_univ'].sel(techid_univ = slice(2,11), region = [8,9,10,26]).sum().values/e['co2e_univ2'].sel(region = [8,9,10,26]).sum().values)

#Electricity use in dat.tf
print("Africa Electricity use univ in dat.tf in TWh: ", e['elec_use_r'].sel(region = [8,9,10,26]).sum().values/1e9)

print('the share of renewables for dat.tf BL Africa is: ' , e['renshare_univ'].sel(region = [8,9,10,26]).mean().values*100)

afr = gph.popg.tpop.where((gph.popg.imager == 8) | (gph.popg.imager == 9) | (gph.popg.imager== 10) | (gph.popg.imager == 26), drop = True)

CG = afr.sel(year = dat.target_year).where((techid_univ==1)| (techid_univ==2)| (techid_univ==3)).sum().values/1e6
SAMG = afr.sel(year = dat.target_year).where(techid_univ>3).sum().values/1e6
print('Africa million people connected to CG:' , CG)
print('Africa million people connected to offgrid:' , SAMG)

#Saving emissions UA
e = e[['co2e_univ2', 'renshare_univ']]
e.to_netcdf('..\\output\\co2emissions_univ_kg.nc')

In [ ]:
#Plotting distribution of least cost options in the population for achieving universal access
year = es.s.year.data

arr = techid_univ.to_dataset(name = 'leastcostID_enteryear_univ')
arr['leastcostID_enteryear_univ'] = xr.where(arr.leastcostID_enteryear_univ == 1, np.nan, arr.leastcostID_enteryear_univ)
arr['pop'] = gph.popg.tpop.sel(year = dat.tf)  #.tpop.sel(year = dat.tf)
df1 = pd.DataFrame([])
for r in [4,6, 8.0, 9.0,10,21,22,24,25,26]:
    df = arr.where(arr.imager == r)
    df = df.pop.groupby(df.leastcostID_enteryear_univ).sum().drop(['spatial_ref',  'dos', 'year', 'seis', 'tres']).to_dataframe(dim_order = ['leastcostID_enteryear_univ'], name = 'pop')
    df['region'] = r
    df['population_share'] = df['pop'] / df['pop'].sum()
    df1 = pd.concat([df1,df])#df1.append(df)
df1

df1 =df1.reset_index()
df1 = df1.replace({'leastcostID_enteryear_univ': {  2:'CG-densification', 3: 'CG-extension', 4: 'MG-PV', 5: 'MG-diesel', 6: 'MG-wind', 7:'MG-hydro' , 8:'MG-PV-diesel', 9:'MG-wind-diesel', 10:'SA-PV', 11: "SA-diesel"}}) #1: 'already access',
df1 = df1.replace({'region':  {4: 'Rest Central America', 6:'Rest South America', 8:'Western Africa', 9: 'Eastern Africa', 10: 'South Africa', 21: 'Southeastern Asia', 22: 'Indonesia +', 24:'Oceania' , 25:'Rest S.Asia', 26:'Rest S.Africa'}})  #{ 8:'Western Africa', 9: 'Eastern Africa', 10: 'South Africa', 26:'Rest SSA'}}) #{6:'Rest South America', 8:'Western Africa', 9: 'Eastern Africa', 10: 'South Africa', 21: 'Southeastern Asia', 22: 'Indonesia +', 24:'Oceania' , 25:'Rest S.Asia', 26:'Rest S.Africa'}})

import plotly.express as px


fig = px.bar(df1,x='region', y='population_share',  color="leastcostID_enteryear_univ", labels = {"leastcostID_enteryear_univ" : "Legend:" , 'population_share': '% population', 'region': 'Region'},
                category_orders={"leastcostID_enteryear_univ" : [ 'CG-densification', 'CG-extension', 'MG-PV',  'MG-wind','MG-hydro' ,  'MG-wind-diesel', 'MG-PV-diesel','MG-diesel', "SA-diesel", 'SA-PV'],
                                    'region':['Rest Central America', 'Rest South America', 'Western Africa',  'Eastern Africa',  'South Africa', 'Rest S.Africa', 'Southeastern Asia',  'Indonesia +', 'Oceania' , 'Rest S.Asia']}) #'already access',#category_orders={"region": [ 8.0, 9.0,10,26]} )

fig.add_scattergl( x = ['Rest Central America', 'Rest South America', 'Western Africa',  'Eastern Africa',  'South Africa', 'Rest S.Africa', 'Southeastern Asia',  'Indonesia +', 'Oceania' , 'Rest S.Asia'], 
                y = [0.97, 0.99, 0.5 , 0.59, 0.84,  0.48, 0.95, 0.79, 0.94, 0.94], line = {'color': 'grey'}, name='Acces level by 2020')
fig.update_layout(font = dict(size = 16), height = 500, width = 1300) #, legend=dict(orientation="h", yanchor="bottom", xanchor="left", ))
